# **SVM Classifier**
*   Implementation of a classic SVM classifier supporting **multi-precision** training.
    *   Implementation currently supports training in either double, single, or half precision.
    *   This implies that both the computations and parameter storage are done in the specified precision.
*   Implementation of a classic SVM network classifier supporting **mixed-precision** training.
    *   Implementation currently supports half precision computations with single precision parameter storage.

In [2]:
import time
import tensorflow as tf
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [3]:
# Set a global random seed
tf.random.set_seed(12)

In [4]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 5

In [5]:
def build_and_train(X_train, y_train, precision):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
        layers.Dense(3, activation='linear', dtype=dtype)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=100)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [6]:
def build_and_train_mixed(X_train, y_train):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    model = models.Sequential([
        layers.Dense(3, activation='linear')
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    tf.keras.mixed_precision.set_global_policy('float32')
    return model, training_time

In [7]:
# Load dataset and split into train and test sets
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(X_train, y_train, precision='single')
_ = build_and_train_mixed(X_train, y_train)

Epoch 1/100
4/4 [==============================] - 2s 6ms/step - loss: 1.5088 - accuracy: 0.2250
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4935 - accuracy: 0.2250
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4796 - accuracy: 0.2333
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4651 - accuracy: 0.2500
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4510 - accuracy: 0.2500
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4369 - accuracy: 0.2667
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4232 - accuracy: 0.2833
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4096 - accuracy: 0.2917
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.3959 - accuracy: 0.2917
Epoch 10/100
4/4 [==============================] - 0s 3ms/step - loss: 1.3828 - accuracy: 0.2917
Epoch 11/100
4/4 [===========

In [9]:
# Train and evaluate with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(X_train, y_train, precision='double')
    time_double += training_time
accuracy_double = model_double.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/100
4/4 [==============================] - 1s 5ms/step - loss: 2.3705 - accuracy: 0.1500
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 2.3517 - accuracy: 0.1500
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 2.3343 - accuracy: 0.1583
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 2.3155 - accuracy: 0.1583
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 2.2977 - accuracy: 0.1583
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 2.2793 - accuracy: 0.1583
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 2.2616 - accuracy: 0.1583
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 2.2441 - accuracy: 0.1583
Epoch 9/100
4/4 [==============================] - 0s 3ms/step - loss: 2.2260 - accuracy: 0.1583
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 2.2087 - accuracy: 0.1583
Epoch 11/100
4/4 [===========

In [22]:
# Train and evaluate with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(X_train, y_train, precision='single')
    time_single += training_time
accuracy_single = model_single.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/100
4/4 [==============================] - 1s 5ms/step - loss: 1.5544 - accuracy: 0.1917
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 1.5402 - accuracy: 0.2000
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 1.5270 - accuracy: 0.2000
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 1.5130 - accuracy: 0.2083
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4996 - accuracy: 0.2167
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4861 - accuracy: 0.2250
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4730 - accuracy: 0.2250
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4602 - accuracy: 0.2333
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4472 - accuracy: 0.2333
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4344 - accuracy: 0.2333
Epoch 11/100
4/4 [===========

In [20]:
# Train and evaluate with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(X_train, y_train, precision='half')
    time_half += training_time
accuracy_half = model_half.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/100
4/4 [==============================] - 0s 7ms/step - loss: 1.1271 - accuracy: 0.2917
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 1.1141 - accuracy: 0.3083
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 1.1027 - accuracy: 0.3167
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0900 - accuracy: 0.3333
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0790 - accuracy: 0.3917
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0667 - accuracy: 0.4417
Epoch 7/100
4/4 [==============================] - 0s 5ms/step - loss: 1.0557 - accuracy: 0.5167
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0450 - accuracy: 0.5333
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0333 - accuracy: 0.5917
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0230 - accuracy: 0.6083
Epoch 11/100
4/4 [===========

In [26]:
# Train with mixed half precision
time_mixed = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_mixed, training_time = build_and_train_mixed(X_train, y_train)
    time_mixed += training_time
accuracy_mixed = model_mixed.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
4/4 [==============================] - 1s 5ms/step - loss: 1.3704 - accuracy: 0.1750
Epoch 2/5
4/4 [==============================] - 0s 4ms/step - loss: 1.3571 - accuracy: 0.1750
Epoch 3/5
4/4 [==============================] - 0s 5ms/step - loss: 1.3440 - accuracy: 0.1750
Epoch 4/5
4/4 [==============================] - 0s 4ms/step - loss: 1.3305 - accuracy: 0.1750
Epoch 5/5
4/4 [==============================] - 0s 4ms/step - loss: 1.3172 - accuracy: 0.1917
Epoch 1/5
4/4 [==============================] - 1s 5ms/step - loss: 0.9805 - accuracy: 0.5250
Epoch 2/5
4/4 [==============================] - 0s 5ms/step - loss: 0.9703 - accuracy: 0.5250
Epoch 3/5
4/4 [==============================] - 0s 5ms/step - loss: 0.9614 - accuracy: 0.5167
Epoch 4/5
4/4 [==============================] - 0s 7ms/step - loss: 0.9522 - accuracy: 0.5167
Epoch 5/5
4/4 [==============================] - 0s 6ms/step - loss: 0.9431 - accuracy: 0.5250
Epoch 1/5
4/4 [==============================] - 1

In [27]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in mixed half precision:", time_mixed/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half precision:", accuracy_half)
print("Accuracy with mixed half precision:", accuracy_mixed)

---RESULTS---
Average training time in double precision: 4.560724544525146 seconds
Average training time in single precision: 3.09025559425354 seconds
Average training time in half precision: 3.025945615768433 seconds
Average training time in mixed half precision: 1.106421661376953 seconds
-------------
Accuracy with double precision: 0.8666666746139526
Accuracy with single precision: 0.800000011920929
Accuracy with half precision: 0.7666666507720947
Accuracy with mixed half precision: 0.7333333492279053
